In [6]:
#Nicholas Chludzinski

import pandas as pd

# Load Counts and Lengths Files
counts = pd.read_csv('/Users/nicho/Downloads/Galaxy45-[featureCounts_on_data_39__Counts].tabular', sep='\t')
lengths = pd.read_csv('/Users/nicho/Downloads/Galaxy47-[featureCounts_on_data_39__Feature_lengths].tabular', sep='\t')

# Input Validation (Quality Check)
# Check for missing values
print("Counts Missing Values:\n", counts.isnull().sum())  
print("\nLengths Missing Values:\n", lengths.isnull().sum())  

# Ensure all GeneIDs in counts are present in lengths (Quality Check)
missing_genes = set(counts['Geneid']) - set(lengths['Geneid'])
# Should be empty
print(f"\nMissing Gene IDs: {missing_genes}")   

# Merge Counts and Lengths on Geneid
data = pd.merge(counts, lengths, on='Geneid')

# Merged Data Validation (Quality Check)
# Check for duplicates
print("\nDuplicates in Merged Data:", data.duplicated(subset='Geneid').sum()) 
# Check negative counts
print("\nNegative Counts:\n", data[data['HISAT2 on data 35: aligned reads (BAM)'] < 0])  
# Check invalid lengths
print("\nNegative or Zero Lengths:\n", data[data['Length'] <= 0])  

# Calculate RPKM (indicates the relative expression level of a gene within a sample)
data['RPKM'] = (data['HISAT2 on data 35: aligned reads (BAM)'] / data['Length']) / data['HISAT2 on data 35: aligned reads (BAM)'].sum() * 1e9

# Calculate TPM (represents gene expression in a way that allows direct comparison across samples)
data['TPM'] = (data['HISAT2 on data 35: aligned reads (BAM)'] / data['Length']) / (data['HISAT2 on data 35: aligned reads (BAM)'] / data['Length']).sum() * 1e6

# Validate RPKM and TPM Calculations (Quality Check)
# Should be close to 1,000,000
print(f"\nTotal TPM: {data['TPM'].sum()}")  
# Check for negative RPKM values
print("\nNegative RPKM Values:\n", data[data['RPKM'] < 0])  
# Check for negative TPM values
print("\nNegative TPM Values:\n", data[data['TPM'] < 0])  

# Top Expressed Genes
top_genes = data.nlargest(10, 'TPM')
print("\nTop 10 Genes by TPM (Higher TPM and RPKM values indicate higher gene expression levels):\n", top_genes[['Geneid', 'TPM', 'RPKM']])

# Summary Statistics for Counts, RPKM, and TPM
print("\nSummary Statistics:\n", data[['HISAT2 on data 35: aligned reads (BAM)', 'RPKM', 'TPM']].describe())

# Save Results
data.to_csv('/Users/nicho/Downloads/gene_expression_results_with_QC.csv', index=False)
print("Results saved with QC checks")


Counts Missing Values:
 Geneid                                    0
HISAT2 on data 35: aligned reads (BAM)    0
dtype: int64

Lengths Missing Values:
 Geneid    0
Length    0
dtype: int64

Missing Gene IDs: set()

Duplicates in Merged Data: 0

Negative Counts:
 Empty DataFrame
Columns: [Geneid, HISAT2 on data 35: aligned reads (BAM), Length]
Index: []

Negative or Zero Lengths:
 Empty DataFrame
Columns: [Geneid, HISAT2 on data 35: aligned reads (BAM), Length]
Index: []

Total TPM: 1000000.0

Negative RPKM Values:
 Empty DataFrame
Columns: [Geneid, HISAT2 on data 35: aligned reads (BAM), Length, RPKM, TPM]
Index: []

Negative TPM Values:
 Empty DataFrame
Columns: [Geneid, HISAT2 on data 35: aligned reads (BAM), Length, RPKM, TPM]
Index: []

Top 10 Genes by TPM (Higher TPM and RPKM values indicate higher gene expression levels):
        Geneid           TPM          RPKM
4047     3043  90880.492380  91503.824527
15266    3514  70101.892150  70582.707799
9286     6176  22730.718524  22886